In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.duplicated()

In [ ]:
y = df['loss']
X = df.drop(['id','loss'], axis=1)
X

# PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(X)

In [ ]:
pca = PCA(n_components=78)
pca.fit(X)
X_transformed = pca.transform(X)

In [ ]:
X_transformed = pd.DataFrame(X_transformed)
X_transformed

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, random_state=10)

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=50, criterion='mse', max_depth=10, random_state=10)
rfr.fit(X_train, y_train)
rfr.score(X_test, y_test)

In [ ]:
df_test_normalized = df_test.drop('id', axis=1)
pca.fit(df_test_normalized)
test_transformed = pca.transform(df_test_normalized)
y_pred = rfr.predict(test_transformed)

In [ ]:
df_sub = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')
#df_sub['loss'] = y_pred
#df_sub.to_csv('submission.csv', index = False)
df_sub.head(10)

# LGBM

In [ ]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(boosting_type='gbdt', n_estimators=100, objective='regression', random_state=10)
lgbm.fit(X_train, y_train)
lgbm.score(X_test, y_test)

In [ ]:
X_test_submit = df_test.drop('id', axis=1)
pca = PCA(n_components=78)
pca.fit(X_test_submit)
test_transformed = pca.transform(X_test_submit)
y_pred = lgbm.predict(test_transformed)

In [ ]:
#df_sub['loss'] = y_pred
#df_sub.to_csv('submission.csv' , index = False)
df_sub.head(10)

# Extreme Learning Machine

In [ ]:
def elm_fit(X, target, h, W=None):
  start_time = time.time()
  
  if W is None:
    W = np.random.uniform(-.5, .5, (h, len(X[0])))
    
  Hinit = X @ W.T
  H = 1 / (1 + np.exp(-Hinit))
  Ht = H.T
  Hp = np.linalg.inv(Ht @ H) @ Ht
  beta = Hp @ target
  y = H @ beta
  mape = sum(abs(y - target) / target) * 100 / len(target)
  
  execution = time.time() - start_time
  print("Waktu eksekusi: %s detik" % execution)
  
  return W, beta, mape

In [ ]:
def elm_predict(X, W, b, round_output=False):
  Hinit = X @ W.T
  H = 1 / (1 + np.exp(-Hinit))
  y = H @ b
  
  if round_output:
    y = [int(round(x)) for x in y]
  
  return y

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_transformed)
#X_normalized = minmax_scale(X_transformed)
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=.3)

In [ ]:
W, b, mape = elm_fit(X_train, y_train, 1000)
print('MAPE:', mape)
output = elm_predict(X_test, W, b, round_output=True)
accuracy = accuracy_score(output, y_test)

#print('Output:', output)
print('True :', y_test)
print('Accuracy:', accuracy)

In [ ]:
pca = PCA(n_components=70)
df_test_drop = df_test.drop('id', axis=1)
pca.fit(df_test_drop)
df_test_transformed = pca.transform(df_test_drop)
df_test_normalized = scaler.fit_transform(df_test_transformed)

y_pred = elm_predict(df_test_normalized, W, b, round_output=False)

In [ ]:
df_sub = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')
df_sub['loss'] = y_pred
df_sub.to_csv('submission.csv' , index = False)
df_sub.head(10)